# Image Classification Dataset COCO untuk Kategori Anjing dan Kucing

Aditya Rachman Putra  
Harits Abdurrohman

Data Checkpoint, saved model, dan raw features bisa diakses di [VM](https://3a4285ffc4f4b3f1-dot-asia-southeast1.notebooks.googleusercontent.com/lab/tree/modified-FCOS/DTL) atau bisa mengontak adityarputra@gmail.com atau harits.abrd@gmail.com

## Dataset COCO

Merupakan dataset untuk Object Detection dengan banyak category, namun untuk tugas ini disederhanakan menjadi suatu masalah klasifikasi gambar dengan dua kategori (binary classification). Eksperimen yang dilakukan secara garis besar dibagi menjadi dua bagian, yaitu eksperimen representasi data gambar (dan secara tidak langsung juga feature engineering). Dan eksperimen parameter model yang digunakan.

karena hanya ada dua kelas, gambar yang digunakan merupakan gambar yang berisi salah satu kelas target saja (anjing atau kucing) namun tidak keduanya.

## Experiment Parameter Model

### Decision Tree Learning

- Maximum tree Depth : 10, 100, 1000
- Criterion : Gini, Entropy

### Random Forest dan XGBoost

- Maximum Depth : 4, 6, 10
- Learning Rate : 0.1 dan 0.3
- Jumlah Estimators : 1, 5, 10, 50


## Eksperimen Representasi

- PCA
- SVD
- Texture
- Color Moment



# Setup COCO API


In [1]:
!pip install Cython
!pip install -q "tqdm>=4.36.1"
!pip3 install xgboost
!pip3 install scikit-image scikit-learn graphviz

In [2]:
!git clone https://github.com/waleedka/coco
!pip install -U setuptools
!pip install -U wheel
!make install -C coco/PythonAPI

fatal: destination path 'coco' already exists and is not an empty directory.
Requirement already up-to-date: setuptools in /opt/conda/lib/python3.7/site-packages (50.3.2)
Requirement already up-to-date: wheel in /opt/conda/lib/python3.7/site-packages (0.35.1)
make: Entering directory '/home/jupyter/modified-FCOS/DTL/coco/PythonAPI'
# install pycocotools to the Python site-packages
python setup.py build_ext install
running build_ext
building 'pycocotools._mask' extension
creating build
creating build/temp.linux-x86_64-3.7
creating build/temp.linux-x86_64-3.7/pycocotools
creating build/common
gcc -pthread -B /opt/conda/compiler_compat -Wl,--sysroot=/ -Wsign-compare -DNDEBUG -g -fwrapv -O3 -Wall -Wstrict-prototypes -fPIC -I/opt/conda/lib/python3.7/site-packages/numpy/core/include -I../common -I/opt/conda/include/python3.7m -c pycocotools/_mask.c -o build/temp.linux-x86_64-3.7/pycocotools/_mask.o -Wno-cpp -Wno-unused-function -std=c99
^C
interrupted
Makefile:7: recipe for target 'install' 

In [3]:
!wget http://images.cocodataset.org/annotations/annotations_trainval2017.zip
!unzip ./annotations_trainval2017.zip

--2020-11-06 01:44:33--  http://images.cocodataset.org/annotations/annotations_trainval2017.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 52.216.88.163
Connecting to images.cocodataset.org (images.cocodataset.org)|52.216.88.163|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 252907541 (241M) [application/zip]
Saving to: ‘annotations_trainval2017.zip’

      annotations_t   0%[                    ] 223.26K   263KB/s               ^C
Archive:  ./annotations_trainval2017.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of ./annotations_trainval2017.zip or
        ./annotations_trainval2017.zip.zip, and cannot find ./annotations_trainval2017.zip.ZIP, period.


In [31]:
!wget http://images.cocodataset.org/annotations/image_info_test2017.zip
!unzip ./image_info_test2017.zip

--2020-11-11 00:58:36--  http://images.cocodataset.org/annotations/image_info_test2017.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 52.217.103.84
Connecting to images.cocodataset.org (images.cocodataset.org)|52.217.103.84|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1144034 (1.1M) [application/zip]
Saving to: ‘image_info_test2017.zip’

image_info_test2017 100%[===================>]   1.09M   873KB/s    in 1.3s    

2020-11-11 00:58:38 (873 KB/s) - ‘image_info_test2017.zip’ saved [1144034/1144034]

Archive:  ./image_info_test2017.zip
  inflating: annotations/image_info_test-dev2017.json  
  inflating: annotations/image_info_test2017.json  


# Import needed Packages and Load COCO categories

In [2]:
from pycocotools.coco import COCO
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, PredefinedSplit, train_test_split
from skimage.transform import rescale, resize, downscale_local_mean
from skimage.util import crop
from skimage import io, color
from math import floor, ceil
from PIL import Image
from joblib import dump, load
from tqdm.notebook import tqdm
from os import path
import xgboost as xgb
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import graphviz 
import json
import pandas as pd

In [3]:
def split_integer(num):
    return (floor(num/2), ceil(num/2))

def load_resize_crop_img(img_url, target_width=64, rgb = True):
    I = io.imread(img_url)
    factor = target_width/min(I.shape[0:2])
    resize_target = (round(I.shape[0] * factor), round(I.shape[1] * factor))
    resized_image = (resize(I, resize_target, anti_aliasing=False) * 255).astype(np.uint8)
    crop_width = (split_integer(resize_target[0] - target_width), split_integer(resize_target[1] - target_width), (0,0))
    if rgb:
        cropped_image = crop(color.gray2rgb(resized_image), crop_width)
    else:
        cropped_image = crop(color.rgb2gray(resized_image), crop_width)
    return cropped_image

def get_coco_id(annotation_file, cat1 = "dog", cat2 = "cat", n = None):
    coco = COCO(annotation_file)
    catIds = coco.getCatIds(catNms=[cat1,cat2])
    img_combine = coco.getImgIds(catIds=catIds)
    img_cat1 = list(set(coco.getImgIds(catIds=coco.getCatIds(catNms=[cat1]))) - set(img_combine))
    img_cat2 = list(set(coco.getImgIds(catIds=coco.getCatIds(catNms=[cat2]))) - set(img_combine))
    if n is None:
        return img_cat1 + img_cat2, np.array(len(img_cat1) * [1] + len(img_cat2) * [0])
    return img_cat1[:n] + img_cat2[:n], np.array(len(img_cat1[:n]) * [1] + len(img_cat2[:n]) * [0])

In [10]:
train_path="annotations/instances_train2017.json"
val_path="annotations/instances_val2017.json"
test_path="annotations/instances_val2017.json"
annotation = {"train":train_path, "val":val_path}

N_TRAIN = 4000
IMG_SIZE = 64

# Get COCO Image Data

Untuk mempermudah melakukan eksperimen berulang, data gambar COCO di simpan pada storage mesin. Dan gambar akan di load kedalam variable sesuai kebutuhan. Dan karena COCO sudah memiliki train-val split sendiri.

Untuk eksperimen awal, digunakan representasi yang "naive" dari data gambar, yaitu diubah menjadi suatu flat array dengan ukuran 1 x (size x size x 3) dengan size yang digunakan sebesar 64 pixel (karena keterbatasan RAM dan grid search tidak support partial_fit maka digunakan ukuran gambar yang masih _muat_ kedalam Memory). Untuk gambar yang dalam format Grayscale, akan diubah menjadi RGB dengan SKImage Gray2RGB.

> **What's Next**
> - Bandingkan hasil antara image yang RGB dengan grayscale seperti apa
> - Mencari cara untuk bisa melakukan training tanpa terlalu terbatas pada ukuran RAM seperti dengan menggunakan Numpy 
> - Coba untuk Train-Val split menggunakan SKlearn train-test-split dengan test 1/3 dari keseluruhan data

In [6]:
X_train, y_train = get_coco_id(annotation["train"], n=N_TRAIN)
X_val, y_val = get_coco_id(annotation["val"])

loading annotations into memory...
Done (t=20.28s)
creating index...
index created!
loading annotations into memory...
Done (t=0.65s)
creating index...
index created!


In [16]:
coco = COCO(annotation["train"])
for x in tqdm(X_train):
    fn = "train/{}.png".format(x)
    if path.exists(fn):
#         print(f"skipping {x}")
        continue
    I = io.imread(coco.loadImgs(x)[0]['coco_url'])
    io.imsave(fn,I)

loading annotations into memory...
Done (t=18.34s)
creating index...
index created!


In [17]:
coco = COCO(annotation["val"])
for x in tqdm(X_val):
    fn = "val/{}.png".format(x)
    if path.exists(fn):
#         print(f"skipping {x}")
        continue
    I = io.imread(coco.loadImgs(x)[0]['coco_url'])
    io.imsave(fn,I)

loading annotations into memory...
Done (t=1.89s)
creating index...
index created!


## Preprocess the image and flatten it

In [55]:
# coco = COCO(annotation["train"])
X_loaded = []
for x in tqdm(X_train):
    try : 
        fn = "train/{}.png".format(x)
        X_loaded += [load_resize_crop_img(fn, target_width=IMG_SIZE).flatten()]
    except :
        print(x)
print("Finished with length: ",len(X_loaded))
X_loaded = np.array(X_loaded)
# df_train = pd.Dataframe({"X":X_loaded,"y":y_train})
print("Converted to ndarray with shape: ",X_loaded.shape, y_train.shape)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:19: FutureWarning: Pass-through of possibly RGB images in gray2rgb is deprecated. In version 0.19, input arrays will always be considered grayscale, even if the last dimension has length 3 or 4. To prevent this warning and ensure compatibility with future versions, detect RGB images outside of this function.



Finished with length:  8083
Converted to ndarray with shape:  (8083, 12288) (8083,)


In [56]:
# coco = COCO(annotation["val"])
X_loaded_val = []
for x in tqdm(X_val):
    try : 
        fn = "val/{}.png".format(x)
        X_loaded_val += [load_resize_crop_img(fn, target_width=IMG_SIZE).flatten()]
    except :
        print(x)
print("Finished with length: ",len(X_loaded_val))
X_loaded_val = np.array(X_loaded_val)
print("Converted to ndarray with shape: ",X_loaded_val.shape, y_val.shape)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:19: FutureWarning: Pass-through of possibly RGB images in gray2rgb is deprecated. In version 0.19, input arrays will always be considered grayscale, even if the last dimension has length 3 or 4. To prevent this warning and ensure compatibility with future versions, detect RGB images outside of this function.



Finished with length:  337
Converted to ndarray with shape:  (337, 12288) (337,)


## Prepare the fold for train-validate GridSearch

Seperti yang sudah dibahas sebelumnya, untuk saat ini digunakan pembagian data training dan validasi mengikuti pembagian pada dataset COCO, sehingga didapatkan jumlah data training di 8083 gambar dan validasi di 337 gambar. Karena jumlah gambar validasi ini relatif sedikit, seharusnya bisa diubah dengan menggunakan train-test-split dari sklearn sehingga pembagian dari train dan validasi lebih _imbang_.

In [8]:
test_fold = np.concatenate([
    # The training data.
    np.full(len(X_train), -1, dtype=np.int8),
    # The development data.
    np.zeros(len(X_val), dtype=np.int8)
])
cv = PredefinedSplit(test_fold)

X = np.concatenate([X_loaded, X_loaded_val])
y = np.concatenate([y_train, y_val])

# dtrain = xgb.DMatrix(X_loaded, label=y_train)
# dval = xgb.DMatrix(X_loaded_val, label=y_val)

((337,), (8083,))

### Save features and label for use later

Data numpy array dari flattened array disimpan kedalam npy file agar bisa digunakan kembali tanpa perlu melakukan preprocess dari gambar, dan variable yang sudah tidak digunakan dan mengambil ruang memory dihapus terlebih dahulu sebelum dimulai fitting

In [47]:
# np.save("coco_dog_cat_feature_{}_{}.npy".format(N_TRAIN, IMG_SIZE), X)
# np.save("coco_dog_cat_label_{}_{}.npy".format(N_TRAIN, IMG_SIZE), y)

(5894, 12288) (1263, 12288) (1263, 12288)


In [73]:
# Load the saved data
X = np.load("coco_dog_cat_feature_{}_{}.npy".format(4000, IMG_SIZE))
y = np.load("coco_dog_cat_label_{}_{}.npy".format(4000, IMG_SIZE))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=101)

test_fold = np.concatenate([
    # The training data.
    np.full(len(X_train), -1, dtype=np.int8),
    # The development data.
    np.zeros(len(X_val), dtype=np.int8)
])
cv = PredefinedSplit(test_fold)

X = np.concatenate([X_train, X_val])
y = np.concatenate([y_train, y_val])

print(X_train.shape, X_val.shape, X_test.shape)

(5894, 12288) (1263, 12288) (1263, 12288)


In [61]:
# Magic to delete unneeded variable (in this case the features array and label array) as it has been aggregated to X and y respectively
# %reset_selective "^X_loaded(_train)?$"
# %reset_selective "^y_(train|val)$"

Once deleted, variables cannot be recovered. Proceed (y/[n])?   y
Once deleted, variables cannot be recovered. Proceed (y/[n])?   y


# Grid Search Model Parameter

Pada bagian ini dilakukan grid search pada setiap model dengan parameter yang telah ditentukan sebelumnya untuk menemukan kombinasi parameter yang memberi akurasi rata-rata terbaik. Dan digunakan scorer default dari masing-masing model, atau dalam kasus ini merupakan [mean_accuracy dari model tersebut ](https://scikit-learn.org/stable/modules/generated/sklearn.base.ClassifierMixin.html)

> **Whats Next**
> - Explore scoring yang sesuai untuk task binary classification yang lebih informatif (e.g. F1-score, AUC-ROC, etc.)
> - Gunakan SKlearn classification_report untuk mendapatkan report yang lebih komprehensif

## Decision Tree Learning 

In [53]:
base = DecisionTreeClassifier()
grid = {
"max_depth" : [10, 100, 1000],
"criterion" : ["gini", "entropy"]
}
model = GridSearchCV(base, grid, cv=cv,n_jobs=4, verbose=100, return_train_score = True)

In [54]:
model = model.fit(X,y)

Fitting 1 folds for each of 6 candidates, totalling 6 fits
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   1 tasks      | elapsed:  1.3min
[Parallel(n_jobs=4)]: Done   2 out of   6 | elapsed:  1.9min remaining:  3.8min
[Parallel(n_jobs=4)]: Done   3 out of   6 | elapsed:  1.9min remaining:  1.9min
[Parallel(n_jobs=4)]: Done   4 out of   6 | elapsed:  1.9min remaining:   57.5s
[Parallel(n_jobs=4)]: Done   6 out of   6 | elapsed:  3.7min remaining:    0.0s
[Parallel(n_jobs=4)]: Done   6 out of   6 | elapsed:  3.7min finished


In [74]:
model = load('gs_dtl_{}_{}.pkl'.format(X.shape[0], IMG_SIZE))

In [75]:
pd.concat([pd.DataFrame(model.cv_results_["rank_test_score"], columns=["Rank"]), 
           pd.DataFrame(model.cv_results_["params"]),
           pd.DataFrame(model.cv_results_["mean_test_score"], columns=["Accuracy"]),
           pd.DataFrame(model.cv_results_["mean_train_score"], columns=["Train_Accuracy"])],axis=1)

,Rank,criterion,max_depth,Accuracy,Train_Accuracy
0,1,gini,10,0.552652,0.873770
1,4,gini,100,0.536025,1.000000
2,3,gini,1000,0.547902,1.000000
3,2,entropy,10,0.551861,0.817102
4,5,entropy,100,0.528108,1.000000
5,6,entropy,1000,0.524149,1.000000


In [76]:
predictions = model.predict(X_test)
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.55      0.55      0.55       612
           1       0.58      0.58      0.58       651

    accuracy                           0.56      1263
   macro avg       0.56      0.56      0.56      1263
weighted avg       0.56      0.56      0.56      1263



In [58]:
dump(model, 'gs_dtl_{}_{}.pkl'.format(X.shape[0], IMG_SIZE))

['gs_dtl_7157_64.pkl']

In [ ]:
# %reset_selective "^model$"

Bisa diamati bahwa untuk decision tree classifier, hasil terbaik untuk dataset ini bisa didapatkan dengan `max_depth = 10` dan `criterion = gini`

## Random Forest Classifier

In [38]:
rf_base = xgb.XGBRFClassifier()
rf_grid = {
"max_depth" : [4, 6, 10],
"learning_rate" : [0.01, 0.1, 0.3],
"n_estimators" : [1, 5, 10, 50]
}
rf_model = GridSearchCV(rf_base, rf_grid, cv=cv,n_jobs=4, verbose=100, return_train_score = True)

In [39]:
rf_model = rf_model.fit(X, y)

Fitting 1 folds for each of 36 candidates, totalling 36 fits
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   1 tasks      | elapsed:   20.6s
[Parallel(n_jobs=4)]: Done   2 tasks      | elapsed:   36.4s
[Parallel(n_jobs=4)]: Done   3 tasks      | elapsed:   40.7s
[Parallel(n_jobs=4)]: Done   4 tasks      | elapsed:   56.6s
[Parallel(n_jobs=4)]: Done   5 tasks      | elapsed:  1.4min
[Parallel(n_jobs=4)]: Done   6 tasks      | elapsed:  1.9min
[Parallel(n_jobs=4)]: Done   7 tasks      | elapsed:  2.1min
[Parallel(n_jobs=4)]: Done   8 tasks      | elapsed:  3.3min
[Parallel(n_jobs=4)]: Done   9 tasks      | elapsed:  3.7min
[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:  3.9min
[Parallel(n_jobs=4)]: Done  11 tasks      | elapsed:  4.5min
[Parallel(n_jobs=4)]: Done  12 tasks      | elapsed:  4.7min
[Parallel(n_jobs=4)]: Done  13 tasks      | elapsed:  5.4min
[Parallel(n_jobs=4)]: Done  14 tasks      | elapsed:  5.7min
[Parallel(

In [77]:
rf_model = load('gs_rf_{}_{}.pkl'.format(X.shape[0], IMG_SIZE))

In [78]:
pd.concat([pd.DataFrame(rf_model.cv_results_["rank_test_score"], columns=["Rank"]), 
           pd.DataFrame(rf_model.cv_results_["params"]),
           pd.DataFrame(rf_model.cv_results_["mean_test_score"], columns=["Accuracy"]),
           pd.DataFrame(rf_model.cv_results_["mean_train_score"], columns=["Train Accuracy"]),
           pd.DataFrame(rf_model.cv_results_["mean_fit_time"], columns=["Fit Time"])],axis=1)

,Rank,learning_rate,max_depth,n_estimators,Accuracy,Train Accuracy,Fit Time
0,28,0.01,4,1,0.589074,0.624533,16.138585
1,25,0.01,4,5,0.590657,0.671700,32.245691
2,19,0.01,4,10,0.594616,0.686461,51.663899
3,7,0.01,4,50,0.605701,0.692569,214.060637
4,31,0.01,6,1,0.578781,0.674584,16.964894
5,22,0.01,6,5,0.592241,0.778758,45.964429
6,16,0.01,6,10,0.596200,0.800136,81.059108
7,1,0.01,6,50,0.609660,0.838819,372.223656
8,34,0.01,10,1,0.551861,0.795385,24.689696
9,13,0.01,10,5,0.600950,0.928062,82.745449


In [41]:
predictions = rf_model.predict(X_test)
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.58      0.54      0.56       612
           1       0.60      0.64      0.62       651

    accuracy                           0.59      1263
   macro avg       0.59      0.59      0.59      1263
weighted avg       0.59      0.59      0.59      1263



In [59]:
dump(rf_model, 'gs_rf_{}_{}.pkl'.format(X.shape[0], IMG_SIZE))

['gs_rf_7157_64.pkl']

In [ ]:
# %reset_selective "^rf_model$"

Untuk Random Forest ini bisa diamati bahwa `learning_rate` tidak mempengaruhi akurasi dari model, namun berpengaruh pada fit time, terutama untuk model dengna jumlah estimator dan tree depth yang tinggi.

## XGBoost Classifier

In [42]:
xg_base = xgb.XGBClassifier()
xg_grid = {
"max_depth" : [4, 6, 10],
"learning_rate" : [0.1, 0.3],
"n_estimators" : [1, 5, 10, 50]
}
xg_model = GridSearchCV(xg_base, xg_grid, cv=cv, n_jobs=4, verbose=100, return_train_score = True)

In [43]:
xg_model = xg_model.fit(X, y)

Fitting 1 folds for each of 24 candidates, totalling 24 fits
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   1 tasks      | elapsed:   17.8s
[Parallel(n_jobs=4)]: Done   2 tasks      | elapsed:   39.7s
[Parallel(n_jobs=4)]: Done   3 tasks      | elapsed:   40.4s
[Parallel(n_jobs=4)]: Done   4 tasks      | elapsed:  1.1min
[Parallel(n_jobs=4)]: Done   5 tasks      | elapsed:  1.6min
[Parallel(n_jobs=4)]: Done   6 tasks      | elapsed:  2.2min
[Parallel(n_jobs=4)]: Done   7 tasks      | elapsed:  2.4min
[Parallel(n_jobs=4)]: Done   8 tasks      | elapsed:  4.0min
[Parallel(n_jobs=4)]: Done   9 tasks      | elapsed:  4.2min
[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:  4.5min
[Parallel(n_jobs=4)]: Done  11 tasks      | elapsed:  5.1min
[Parallel(n_jobs=4)]: Done  12 tasks      | elapsed:  5.8min
[Parallel(n_jobs=4)]: Done  13 tasks      | elapsed:  6.1min
[Parallel(n_jobs=4)]: Done  14 tasks      | elapsed:  6.5min
[Parallel(

In [79]:
xg_model = load('gs_xg_{}_{}.pkl'.format(X.shape[0], IMG_SIZE))

In [80]:
pd.concat([pd.DataFrame(xg_model.cv_results_["rank_test_score"], columns=["Rank"]), 
           pd.DataFrame(xg_model.cv_results_["params"]),
           pd.DataFrame(xg_model.cv_results_["mean_test_score"], columns=["Accuracy"]),
           pd.DataFrame(xg_model.cv_results_["mean_train_score"], columns=["Train Accuracy"]),
           pd.DataFrame(xg_model.cv_results_["mean_fit_time"], columns=["Fit Time"])],axis=1)

,Rank,learning_rate,max_depth,n_estimators,Accuracy,Train Accuracy,Fit Time
0,14,0.1,4,1,0.577989,0.628775,14.644902
1,11,0.1,4,5,0.594616,0.686630,35.326828
2,5,0.1,4,10,0.610451,0.717170,58.580081
3,2,0.1,4,50,0.625495,0.844757,244.962860
4,20,0.1,6,1,0.571655,0.697489,18.858478
5,16,0.1,6,5,0.577197,0.811503,55.282102
6,8,0.1,6,10,0.598575,0.857143,99.300498
7,1,0.1,6,50,0.629454,0.975908,427.975024
8,23,0.1,10,1,0.535234,0.869528,29.053973
9,16,0.1,10,5,0.577197,0.966916,105.474634


In [72]:
predictions = xg_model.predict(X_test)
print(classification_report(y_test, predictions))

predictions = xg_model.predict(X_train)
print(classification_report(y_train, predictions)) 

              precision    recall  f1-score   support

           0       0.59      0.60      0.60       612
           1       0.62      0.61      0.61       651

    accuracy                           0.60      1263
   macro avg       0.60      0.60      0.60      1263
weighted avg       0.60      0.60      0.60      1263

              precision    recall  f1-score   support

           0       0.98      0.96      0.97      2866
           1       0.96      0.98      0.97      3028

    accuracy                           0.97      5894
   macro avg       0.97      0.97      0.97      5894
weighted avg       0.97      0.97      0.97      5894



In [60]:
dump(xg_model, 'gs_xg_{}_{}.pkl'.format(X.shape[0], IMG_SIZE))

['gs_xg_7157_64.pkl']

Bisa diamati bahwa XGBoost memiliki mean accuracy yang lebih tinggi dibandingkan dua model lainnya. Yang dihasilkan oleh parameter learning rate, tre depth, dan jumlah estimator respectively 0.1, 10, 50 dengan mean accuracy 0.649852

# Kesimpulan

Telah dilakukan eksperimen dengan berbagai kombinasi parameter. Jika kita melihat performa terbaik yang dihasilkan per model, bisa diamati bahwa model DTL memberikan performa terburuk dan mendekati performa bila komputer menebak kelas dengan uniformly random (mean accuration 0.558). Lalu untuk Random Forest memiliki performa yang relatif lebih baik dengan mean accuration 0.623. Dan XGBoost memberikan performa terbaik dengna mean accuration 0.649.